In [53]:
import pandas as pd
import numpy as np
import keras

from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

import keras
from keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
def prepare_data():
    # Prepare data into the expected format to the model

    # Read data
    df_ = pd.read_csv('../data3/undersampled_max.csv', index_col=0)

    #Clean data
    df_.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_ = df_.dropna()
    df_ = df_.reset_index()

    #Obtain input and tarfet
    X=df_.drop(['label','index'], axis=1)
    X = df_[['VV amplitude (linear)','VH amplitude (linear)','VV coherence']]
    Y=df_['label']

    #Convert input into desired format
    X = np.array(X)

    # Convert integers to dummy variables (i.e. one hot encoded)
    dummy_y = np_utils.to_categorical(Y)
    return X,dummy_Y

In [23]:
def model(X):

    # build a model
    model_6 = Sequential()
    model_6.add(Dense(128, input_shape=(X.shape[1],), activation='relu')) # input shape is (features,)
    model_6.add(Dense(units=64, kernel_initializer='uniform', activation='tanh'))
    model_6.add(Dense(units=32, kernel_initializer='uniform', activation='tanh'))
    model_6.add(Dense(units=16, kernel_initializer='uniform', activation='tanh'))
    model_6.add(Dropout(0.25))
    model_6.add(Dense(units=8, kernel_initializer='uniform', activation='tanh'))
    model_6.add(Dropout(0.5))
    model_6.add(Dense(4, activation='softmax'))
    model_6.summary()

    # compile the model
    model_6.compile(optimizer='adam', 
                  loss='categorical_crossentropy', # this is different instead of binary_crossentropy (for regular classification)
                  metrics=['accuracy'])
    return model_6

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 128)               512       
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 16)                528       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_8 (Dense)             (None, 8)                 136       
                                                                 
 dropout_1 (Dropout)         (None, 8)                

In [ ]:
def training(x,dummy_y,model_6):
    # Training the model 
    es = keras.callbacks.EarlyStopping(monitor='val_loss', 
                                       mode='min',
                                       restore_best_weights=True) # important - otherwise you just return the last weigths...
    mcp_save =keras.callbacks.ModelCheckpoint('model_6_max.hdf5', save_best_only=True, monitor='accuracy', mode='max')

    # now we just update our model fit call
    history = model_6.fit(X,
                        dummy_y,
                        epochs=1500, # you can set this to a big number!
                        batch_size=100,
                        shuffle=True,
                        validation_split=0.2,
                        verbose=1,
                        callbacks=[mcp_save])
    
    return history,model_6

In [50]:
def metrics(X,dummy_y,model_6)
    #Get model metrics
    preds = model_6.predict(X) # see how the model did!
    print(preds[0]) # i'm spreading that prediction across three nodes and they sum to 1
    print(np.sum(preds[0])) # sum it up! Should be 1
    ## [9.9999988e-01 1.3509347e-07 6.7064638e-16]
    ## 1.0

    # Almost a perfect prediction
    # actual is left, predicted is top
    # names can be found by inspecting Y
    matrix = confusion_matrix(dummy_y.argmax(axis=1), preds.argmax(axis=1))
    matrix
    ## array([[50,  0,  0],
    ##        [ 0, 46,  4],
    ##        [ 0,  1, 49]])


    # more detail on how well things were predicted
    print(classification_report(dummy_y.argmax(axis=1), preds.argmax(axis=1)))

Epoch 1/1500
37/37 [==============================] - 1s 17ms/step - loss: 0.2020 - accuracy: 0.9214 - val_loss: 3.9626 - val_accuracy: 0.3658
Epoch 2/1500
37/37 [==============================] - 0s 7ms/step - loss: 0.2001 - accuracy: 0.9244 - val_loss: 3.9766 - val_accuracy: 0.3801
Epoch 3/1500
37/37 [==============================] - 0s 11ms/step - loss: 0.1920 - accuracy: 0.9247 - val_loss: 3.9553 - val_accuracy: 0.3834
Epoch 4/1500
37/37 [==============================] - 0s 12ms/step - loss: 0.2288 - accuracy: 0.9124 - val_loss: 3.9117 - val_accuracy: 0.3680
Epoch 5/1500
37/37 [==============================] - 0s 8ms/step - loss: 0.4685 - accuracy: 0.8296 - val_loss: 3.9501 - val_accuracy: 0.3516
Epoch 6/1500
37/37 [==============================] - 0s 7ms/step - loss: 0.6122 - accuracy: 0.7935 - val_loss: 3.8963 - val_accuracy: 0.3801
Epoch 7/1500
37/37 [==============================] - 0s 7ms/step - loss: 0.6316 - accuracy: 0.7699 - val_loss: 3.8243 - val_accuracy: 0.3669
Epo

In [ ]:
#Run everything!
X,dummy_Y = prepare_data
model_6 = model(X)
history,model_6 = training(x,dummy_y,model_6)
metrics(X,dummy_y,model_6)